In [31]:
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datasets import load_dataset
import re

## Loading Data

In [27]:
dataset = load_dataset("cnn_dailymail","3.0.0")

Reusing dataset cnn_dailymail (/home/joshcoward63/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0a01b1abede4f646130574f203de57a293ded8a7a11e3406a539453afdfeb2c0)


In [34]:
dataset['test']['article'][0]
# len(dataset['test']['highlights'][0])

'(CNN)James Best, best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV\'s "The Dukes of Hazzard," died Monday after a brief illness. He was 88. Best died in hospice in Hickory, North Carolina, of complications from pneumonia, said Steve Latshaw, a longtime friend and Hollywood colleague. Although he\'d been a busy actor for decades in theater and in Hollywood, Best didn\'t become famous until 1979, when "The Dukes of Hazzard\'s" cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons, Best\'s Rosco P. Coltrane chased the moonshine-running Duke boys back and forth across the back roads of fictitious Hazzard County, Georgia, although his "hot pursuit" usually ended with him crashing his patrol car. Although Rosco was slow-witted and corrupt, Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive "kew-kew-kew" chuckle and for goofy catchphrases such a

## Data Preproccessing 

In [32]:
#The following may not need to be used 
def clean_text(text):
    text = re.sub(r"@[A-Za-z0-9]+", ' ', text)
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text)
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text)
    text = re.sub('\t', ' ',  text)
    text = re.sub(r" +", ' ', text)
    return text

def load_data(path):
    onlyfiles = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    print('found {} files'.format(len(onlyfiles)))
    all_text = []
    for f in onlyfiles:
        with open('{}/{}'.format(path, f)) as handle:
            lines = clean_text(handle.readlines()[0])
            all_text.append(lines)
        
    return all_text

In [33]:
clean_text(dataset['test']['article'][0])

" CNN James Best best known for his portrayal of bumbling sheriff Rosco P. Coltrane on TV's The Dukes of Hazzard died Monday after a brief illness. He was 88. Best died in hospice in Hickory North Carolina of complications from pneumonia said Steve Latshaw a longtime friend and Hollywood colleague. Although he'd been a busy actor for decades in theater and in Hollywood Best didn't become famous until 1979 when The Dukes of Hazzard's cornpone charms began beaming into millions of American homes almost every Friday night. For seven seasons Best's Rosco P. Coltrane chased the moonshine running Duke boys back and forth across the back roads of fictitious Hazzard County Georgia although his hot pursuit usually ended with him crashing his patrol car. Although Rosco was slow witted and corrupt Best gave him a childlike enthusiasm that got laughs and made him endearing. His character became known for his distinctive kew kew kew chuckle and for goofy catchphrases such as cuff 'em and stuff 'em!

In [ ]:
def overlapping_subsection(text):
    for article in dataset['test']['article']:
        word_count = 0
        article_subsections = []
        while len(article) > 512:
            if len(article_subsections) == 0:
                article_subsections.append(article[:512])
                word_count = 512
                article = article[:512]
            if len(article) <= 412:
                article_subsections.append(article[word_count-100:])
            else:
                article_subsections.append(article[word_count-100:word_count+412])   
                word_count = word_count + 412
                article = article[word_count:]
        article_subsections.append(article)
        article = article_subsections

## Create Bert model

In [18]:
import transformers
from transformers import BertTokenizer, BertModel, EncoderDecoderModel
import torch
from tqdm import tqdm_notebook as tqdm


## Encode Data with Bert

In [36]:
#The tokenizer to be used to create embeddings 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased') # initialize Bert2Bert from pre-trained checkpoints

# forward
input_ids = torch.tensor(tokenizer.encode(dataset['test']['article'], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids=input_ids, decoder_input_ids=input_ids)

# # training
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=input_ids)
# loss, logits = outputs.loss, outputs.logits

# # save and load from pretrained
# model.save_pretrained("bert2bert")
# model = EncoderDecoderModel.from_pretrained("bert2bert")

# # generation
# generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer

RuntimeError: The size of tensor a (11492) must match the size of tensor b (512) at non-singleton dimension 1

In [17]:
class BERTClass(torch.nn.Module):
    def __init__(self, NUM_OUT):
        super(BERTClass, self).__init__()
                   
        self.l1 = EncoderDecoderModel.from_encoder_decoder_pretrained("bert-base-uncased","bert-base-uncased")
#         self.pre_classifier = torch.nn.Linear(768, 256)
        self.classifier = torch.nn.Linear(768, NUM_OUT)
#         self.dropout = torch.nn.Dropout(0.5)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
#         pooler = self.pre_classifier(pooler)
#         pooler = torch.nn.Tanh()(pooler)
#         pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.softmax(output)
        return output



In [ ]:
# tokenized_dataset = []
# model = BertClass(NUM_OUT)
# model.to(device)
# for i in range len(dataset['test']):
#     # forward
#     input_ids = torch.tensor(tokenizer.encode(dataset['test']['highlights'][i], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
#     tokenized_dataset.append(model(input_ids=input_ids, decoder_input_ids=input_ids))

# # training
# outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=input_ids)
# loss, logits = outputs.loss, outputs.logits

## Create Bert Decoder

## Train Model

## Validation

## Testing on the Test Set

## Results

## Future Work